# Phase 5: Interpretation & Applications

This notebook demonstrates the interpretation and application capabilities of the MoA prediction framework, including:

1. **Model Explainability & Interpretation**
   - Attention visualization
   - Feature importance analysis
   - Counterfactual explanations
   - Uncertainty estimation

2. **Drug Repurposing Pipeline**
   - MoA-based similarity analysis
   - Candidate ranking and scoring
   - Repurposing hypothesis generation

3. **Knowledge Discovery**
   - Novel drug-pathway associations
   - Biological hypothesis generation
   - Statistical significance testing

4. **Therapeutic Insights**
   - Target identification
   - Drug combination prediction
   - Biomarker discovery
   - Clinical decision support

In [ ]:
import sys
import os
sys.path.append('..')

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# MoA Framework imports
from moa.utils.config import Config
from moa.utils.logger import get_logger
from moa.models.multimodal_model import MultiModalMoAModel
from moa.interpretation.explainer import MoAExplainer
from moa.interpretation.uncertainty import UncertaintyEstimator
from moa.applications.drug_repurposing import DrugRepurposingPipeline
from moa.applications.knowledge_discovery import KnowledgeDiscovery
from moa.applications.therapeutic_insights import TherapeuticInsights

# Set up logging and configuration
logger = get_logger(__name__)
config = Config('../configs/config.yaml')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("Phase 5: Interpretation & Applications")
print("=====================================")

## 1. Model Setup and Demo Data

In [ ]:
# Initialize model
model = MultiModalMoAModel(config)
model.eval()

# Create demo compound data
def create_demo_compound_data(n_compounds=10):
    """Create demonstration compound data."""
    compounds = []
    metadata = []
    
    for i in range(n_compounds):
        # Chemical features
        chemical_features = {
            'node_features': torch.randn(25, 64),
            'edge_index': torch.randint(0, 25, (2, 60)),
            'edge_features': torch.randn(60, 16),
            'batch': torch.zeros(25, dtype=torch.long),
            'counterfactual_weights': torch.rand(25)
        }
        
        # Biological features
        biological_features = {
            'gene_expression': torch.randn(978),
            'pathway_scores': torch.randn(50),
            'mechanism_tokens': torch.randn(128)
        }
        
        # Protein features
        protein_features = {
            'pocket_features': torch.randn(256)
        }
        
        compound_data = {
            'chemical': chemical_features,
            'biological': biological_features,
            'protein': protein_features
        }
        
        compounds.append(compound_data)
        
        # Metadata
        meta = {
            'compound_id': f'DEMO_{i:03d}',
            'compound_name': f'Demo Compound {i+1}',
            'smiles': f'CC(C)C{i}',
            'molecular_weight': 200 + i * 15,
            'drug_class': ['kinase_inhibitor', 'gpcr_agonist', 'ion_channel_blocker'][i % 3],
            'indication': ['cancer', 'diabetes', 'hypertension'][i % 3],
            'development_stage': ['approved', 'clinical', 'preclinical'][i % 3],
            'is_approved_drug': i < 3
        }
        metadata.append(meta)
    
    return compounds, metadata

# Create demo data
demo_compounds, compound_metadata = create_demo_compound_data(15)
moa_classes = [f'MoA_{i}' for i in range(config.model.num_classes)]

print(f"Created {len(demo_compounds)} demo compounds")
print(f"MoA classes: {len(moa_classes)}")

## 2. Model Explainability & Interpretation

In [ ]:
# Initialize explainer
explainer = MoAExplainer(model, config, moa_classes)

# Explain a single prediction
compound_data = demo_compounds[0]
explanation = explainer.explain_prediction(compound_data, top_k_features=10)

print("=== MODEL EXPLANATION ===")
print(f"Compound: {compound_metadata[0]['compound_name']}")
print(f"Prediction confidence: {explanation['prediction_confidence']:.3f}")
print(f"Model certainty: {explanation['model_certainty']:.3f}")

# Display top predictions
print("\nTop Predicted MoAs:")
for i, pred in enumerate(explanation['top_predictions'][:5]):
    print(f"  {i+1}. {pred['moa_name']}: {pred['score']:.3f} ({pred['confidence']})")

# Display modality contributions
print("\nModality Contributions:")
modality_contrib = explanation['modality_contributions']
for modality, contrib in modality_contrib.items():
    print(f"  {modality}: {contrib:.3f}")

# Visualize modality contributions
plt.figure(figsize=(10, 6))
modalities = list(modality_contrib.keys())
contributions = list(modality_contrib.values())

plt.subplot(1, 2, 1)
plt.bar(modalities, contributions, color=['skyblue', 'lightcoral', 'lightgreen'])
plt.title('Modality Contributions')
plt.ylabel('Contribution Score')
plt.xticks(rotation=45)

# Feature importance
feature_importance = explanation['feature_importance'][:10]
feature_names = [f['feature_name'] for f in feature_importance]
importance_scores = [f['importance'] for f in feature_importance]

plt.subplot(1, 2, 2)
plt.barh(range(len(feature_names)), importance_scores)
plt.yticks(range(len(feature_names)), feature_names)
plt.title('Top Feature Importance')
plt.xlabel('Importance Score')

plt.tight_layout()
plt.show()

In [ ]:
# Uncertainty estimation
uncertainty_estimator = UncertaintyEstimator(model, config)

uncertainty_results = uncertainty_estimator.estimate_uncertainty(
    compound_data, n_samples=50
)

print("=== UNCERTAINTY ANALYSIS ===")
print(f"Epistemic uncertainty: {uncertainty_results['epistemic_uncertainty']:.3f}")
print(f"Aleatoric uncertainty: {uncertainty_results['aleatoric_uncertainty']:.3f}")
print(f"Total uncertainty: {uncertainty_results['total_uncertainty']:.3f}")
print(f"Prediction confidence: {uncertainty_results['prediction_confidence']:.3f}")

# Visualize uncertainty
uncertainty_types = ['Epistemic', 'Aleatoric', 'Total']
uncertainty_values = [
    uncertainty_results['epistemic_uncertainty'],
    uncertainty_results['aleatoric_uncertainty'],
    uncertainty_results['total_uncertainty']
]

plt.figure(figsize=(8, 5))
plt.bar(uncertainty_types, uncertainty_values, color=['orange', 'purple', 'red'])
plt.title('Uncertainty Analysis')
plt.ylabel('Uncertainty Score')
plt.ylim(0, max(uncertainty_values) * 1.2)
plt.show()

## 3. Drug Repurposing Pipeline

In [ ]:
# Initialize repurposing pipeline
repurposing_pipeline = DrugRepurposingPipeline(model, config, moa_classes)

# Use first compound as query, others as candidates
query_compound = demo_compounds[0]
candidate_compounds = demo_compounds[1:11]
candidate_metadata = compound_metadata[1:11]

# Identify repurposing candidates
repurposing_results = repurposing_pipeline.identify_repurposing_candidates(
    query_compound_data=query_compound,
    target_disease="Type 2 Diabetes",
    candidate_compounds=candidate_compounds,
    compound_metadata=candidate_metadata
)

print("=== DRUG REPURPOSING ANALYSIS ===")
print(f"Query compound: {compound_metadata[0]['compound_name']}")
print(f"Target disease: {repurposing_results['target_disease']}")
print(f"Candidates analyzed: {len(candidate_compounds)}")
print(f"Repurposing potential: {repurposing_results['summary_statistics']['repurposing_potential']}")

# Display top candidates
print("\nTop Repurposing Candidates:")
for i, candidate in enumerate(repurposing_results['ranked_candidates'][:5]):
    idx = candidate['candidate_index']
    metadata = candidate_metadata[idx]
    print(f"  {i+1}. {metadata['compound_name']}")
    print(f"     Similarity Score: {candidate['ranking_score']:.3f}")
    print(f"     Drug Class: {metadata['drug_class']}")
    print(f"     Development Stage: {metadata['development_stage']}")

# Display hypotheses
print("\nRepurposing Hypotheses:")
for i, hypothesis in enumerate(repurposing_results['hypotheses'][:3]):
    print(f"  {i+1}. {hypothesis['hypothesis_text']}")
    print(f"     Confidence: {hypothesis['confidence_level']}")
    print(f"     Shared MoAs: {len(hypothesis['shared_moas'])}")

In [ ]:
# Visualize repurposing results
candidates = repurposing_results['ranked_candidates'][:8]
compound_names = [candidate_metadata[c['candidate_index']]['compound_name'] for c in candidates]
similarity_scores = [c['ranking_score'] for c in candidates]

plt.figure(figsize=(12, 6))

# Similarity scores
plt.subplot(1, 2, 1)
plt.barh(range(len(compound_names)), similarity_scores, color='lightblue')
plt.yticks(range(len(compound_names)), compound_names)
plt.xlabel('Similarity Score')
plt.title('Repurposing Candidate Rankings')
plt.gca().invert_yaxis()

# Confidence distribution
confidence_scores = repurposing_results['confidence_scores']
confidence_levels = ['High', 'Medium', 'Low']
confidence_counts = [
    confidence_scores['high_confidence_candidates'],
    confidence_scores['medium_confidence_candidates'],
    confidence_scores['low_confidence_candidates']
]

plt.subplot(1, 2, 2)
plt.pie(confidence_counts, labels=confidence_levels, autopct='%1.1f%%', 
        colors=['green', 'orange', 'red'])
plt.title('Confidence Distribution')

plt.tight_layout()
plt.show()

## 4. Knowledge Discovery

In [ ]:
# Initialize knowledge discovery
pathway_annotations = {}
for i, moa in enumerate(moa_classes[:30]):  # Annotate first 30 MoAs
    if i < 10:
        pathway_annotations[moa] = ['apoptosis', 'cell_cycle']
    elif i < 20:
        pathway_annotations[moa] = ['metabolism', 'signaling']
    else:
        pathway_annotations[moa] = ['dna_repair', 'protein_synthesis']

knowledge_discovery = KnowledgeDiscovery(
    model, config, moa_classes, pathway_annotations
)

# Discover novel associations
discovery_results = knowledge_discovery.discover_novel_associations(
    compound_data_list=demo_compounds,
    compound_metadata=compound_metadata,
    known_associations={}  # No known associations for demo
)

print("=== KNOWLEDGE DISCOVERY ===")
print(f"Compounds analyzed: {discovery_results['total_compounds']}")
print(f"Total associations: {discovery_results['discovery_statistics']['total_associations']}")
print(f"Significant associations: {discovery_results['discovery_statistics']['significant_associations']}")

# Display top associations
print("\nTop Validated Associations:")
for i, assoc in enumerate(discovery_results['validated_associations'][:5]):
    print(f"  {i+1}. {assoc.get('property', assoc.get('pathway', 'Unknown'))} -> {assoc.get('moa', 'Unknown MoA')}")
    print(f"     Method: {assoc['discovery_method']}")
    print(f"     P-value: {assoc.get('p_value', 'N/A')}")
    print(f"     Confidence: {assoc.get('confidence', 'medium')}")

# Display biological hypotheses
print("\nBiological Hypotheses:")
for i, hypothesis in enumerate(discovery_results['biological_hypotheses'][:3]):
    print(f"  {i+1}. {hypothesis['hypothesis_text']}")
    print(f"     Type: {hypothesis['hypothesis_type']}")
    print(f"     Confidence: {hypothesis['confidence']}")

In [ ]:
# Visualize discovery statistics
discovery_stats = discovery_results['discovery_statistics']

plt.figure(figsize=(12, 5))

# Associations by method
plt.subplot(1, 2, 1)
methods = list(discovery_stats['associations_by_method'].keys())
counts = list(discovery_stats['associations_by_method'].values())
plt.bar(methods, counts, color=['skyblue', 'lightcoral', 'lightgreen', 'gold'])
plt.title('Associations by Discovery Method')
plt.ylabel('Number of Associations')
plt.xticks(rotation=45)

# Confidence distribution
plt.subplot(1, 2, 2)
confidence_dist = discovery_stats['confidence_distribution']
confidence_labels = list(confidence_dist.keys())
confidence_values = list(confidence_dist.values())
plt.pie(confidence_values, labels=confidence_labels, autopct='%1.1f%%',
        colors=['green', 'orange', 'red'])
plt.title('Discovery Confidence Distribution')

plt.tight_layout()
plt.show()

## 5. Therapeutic Insights

In [ ]:
# Initialize therapeutic insights
therapeutic_insights = TherapeuticInsights(model, config, moa_classes)

# Disease profile
disease_profile = {
    'disease_name': 'Type 2 Diabetes',
    'target_pathways': ['metabolism', 'signaling', 'insulin_pathway'],
    'dysregulated_pathways': ['glucose_metabolism', 'lipid_metabolism'],
    'biomarkers': ['HbA1c', 'glucose', 'insulin'],
    'therapeutic_targets': ['PPARG', 'DPP4', 'SGLT2']
}

# Identify therapeutic targets
target_results = therapeutic_insights.identify_therapeutic_targets(
    disease_profile=disease_profile,
    compound_data_list=demo_compounds,
    compound_metadata=compound_metadata
)

print("=== THERAPEUTIC TARGET ANALYSIS ===")
print(f"Disease: {disease_profile['disease_name']}")
print(f"Pathways analyzed: {len(disease_profile['target_pathways'])}")

# Display promising targets
promising_targets = target_results['promising_targets']
print(f"\nPromising Targets ({len(promising_targets)} found):")
for i, target in enumerate(promising_targets[:3]):
    print(f"  {i+1}. {target['pathway']}")
    print(f"     Target Score: {target['target_score']:.3f}")
    print(f"     Priority: {target['priority']}")
    print(f"     Druggable Compounds: {target['druggable_compounds']}")

# Display recommendations
recommendations = target_results['therapeutic_recommendations']
print(f"\nTherapeutic Recommendations:")
for i, rec in enumerate(recommendations[:3]):
    print(f"  {i+1}. {rec['target_pathway']}")
    print(f"     Recommendation: {rec['recommendation_type']}")
    print(f"     Priority: {rec['priority']}")
    print(f"     Timeline: {rec['timeline']}")
    print(f"     Success Probability: {rec['success_probability']}")

In [ ]:
# Drug combination prediction
combination_results = therapeutic_insights.predict_drug_combinations(
    compound_data_list=demo_compounds[:8],
    compound_metadata=compound_metadata[:8],
    target_disease="Type 2 Diabetes"
)

print("=== DRUG COMBINATION ANALYSIS ===")
combo_stats = combination_results['combination_statistics']
print(f"Combinations analyzed: {combo_stats['total_combinations']}")
print(f"Synergistic combinations: {combo_stats['synergistic_combinations']}")
print(f"High synergy combinations: {combo_stats['high_synergy_combinations']}")

# Display top combinations
synergistic_combos = combination_results['synergistic_combinations']
print(f"\nTop Synergistic Combinations:")
for i, combo in enumerate(synergistic_combos[:3]):
    print(f"  {i+1}. {combo['compound_1_name']} + {combo['compound_2_name']}")
    print(f"     Synergy Score: {combo['combination_score']:.3f}")
    print(f"     Mechanism: {combo['synergy_mechanism']}")
    print(f"     Shared MoAs: {len(combo['shared_moas'])}")

In [ ]:
# Visualize therapeutic insights
plt.figure(figsize=(15, 10))

# Target scores
plt.subplot(2, 3, 1)
if promising_targets:
    target_names = [t['pathway'] for t in promising_targets[:5]]
    target_scores = [t['target_score'] for t in promising_targets[:5]]
    plt.barh(range(len(target_names)), target_scores, color='lightblue')
    plt.yticks(range(len(target_names)), target_names)
    plt.xlabel('Target Score')
    plt.title('Promising Therapeutic Targets')
    plt.gca().invert_yaxis()

# Combination synergy distribution
plt.subplot(2, 3, 2)
if 'score_distribution' in combo_stats:
    score_dist = combo_stats['score_distribution']
    labels = list(score_dist.keys())
    values = list(score_dist.values())
    plt.pie(values, labels=labels, autopct='%1.1f%%')
    plt.title('Combination Synergy Distribution')

# Recommendation types
plt.subplot(2, 3, 3)
if recommendations:
    rec_types = [r['recommendation_type'] for r in recommendations]
    rec_counts = {}
    for rt in rec_types:
        rec_counts[rt] = rec_counts.get(rt, 0) + 1
    
    plt.bar(list(rec_counts.keys()), list(rec_counts.values()), color='lightcoral')
    plt.title('Recommendation Types')
    plt.xticks(rotation=45)

# Target priorities
plt.subplot(2, 3, 4)
if promising_targets:
    priorities = [t['priority'] for t in promising_targets]
    priority_counts = {}
    for p in priorities:
        priority_counts[p] = priority_counts.get(p, 0) + 1
    
    plt.pie(list(priority_counts.values()), labels=list(priority_counts.keys()), 
            autopct='%1.1f%%', colors=['red', 'orange', 'green'])
    plt.title('Target Priority Distribution')

# Combination scores
plt.subplot(2, 3, 5)
if synergistic_combos:
    combo_names = [f"{c['compound_1_name'][:8]}+{c['compound_2_name'][:8]}" for c in synergistic_combos[:5]]
    combo_scores = [c['combination_score'] for c in synergistic_combos[:5]]
    plt.bar(range(len(combo_names)), combo_scores, color='lightgreen')
    plt.xticks(range(len(combo_names)), combo_names, rotation=45)
    plt.ylabel('Synergy Score')
    plt.title('Top Drug Combinations')

# Summary metrics
plt.subplot(2, 3, 6)
summary_metrics = target_results.get('summary_metrics', {})
if summary_metrics:
    metrics = ['Total Pathways', 'High Activity', 'Druggable Compounds']
    values = [
        summary_metrics.get('total_pathways_analyzed', 0),
        summary_metrics.get('high_activity_pathways', 0),
        summary_metrics.get('total_druggable_compounds', 0)
    ]
    plt.bar(metrics, values, color='gold')
    plt.title('Summary Metrics')
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## 6. Summary and Conclusions

In [ ]:
print("=== PHASE 5 SUMMARY ===")
print("\n✅ MODEL INTERPRETATION & EXPLAINABILITY")
print("   • Attention visualization implemented")
print("   • Feature importance analysis across modalities")
print("   • Counterfactual explanations for molecular fragments")
print("   • Uncertainty estimation with epistemic/aleatoric decomposition")

print("\n✅ DRUG REPURPOSING PIPELINE")
print(f"   • Repurposing potential: {repurposing_results['summary_statistics']['repurposing_potential']}")
print(f"   • Top candidates identified: {len(repurposing_results['ranked_candidates'])}")
print(f"   • Hypotheses generated: {len(repurposing_results['hypotheses'])}")
print("   • Network visualization and comprehensive reporting")

print("\n✅ KNOWLEDGE DISCOVERY")
print(f"   • Total associations discovered: {discovery_results['discovery_statistics']['total_associations']}")
print(f"   • Significant associations: {discovery_results['discovery_statistics']['significant_associations']}")
print(f"   • Biological hypotheses: {len(discovery_results['biological_hypotheses'])}")
print("   • Statistical validation and significance testing")

print("\n✅ THERAPEUTIC INSIGHTS")
print(f"   • Promising targets identified: {len(target_results['promising_targets'])}")
print(f"   • Therapeutic recommendations: {len(target_results['therapeutic_recommendations'])}")
print(f"   • Synergistic combinations: {combo_stats['synergistic_combinations']}")
print("   • Clinical decision support and biomarker discovery")

print("\n🎯 KEY ACHIEVEMENTS:")
print("   • Comprehensive model interpretation framework")
print("   • Automated drug repurposing pipeline")
print("   • Novel knowledge discovery capabilities")
print("   • Clinical decision support tools")
print("   • Biomarker discovery platform")
print("   • Therapeutic target identification")
print("   • Drug combination prediction")

print("\n🚀 READY FOR PHASE 6: PUBLICATION & DEPLOYMENT")
print("   • Research publication preparation")
print("   • API deployment for production use")
print("   • Reproducibility package for community")
print("   • Performance benchmarking on real datasets")
print("   • Clinical validation partnerships")